### Run Asset Pool 1

In [1]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV, lang='english',check=False)

Connecting engine server -> https://absbox.org/api/dev

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅Connected, local lib:0.46.5, server:0.50.1

#### Pool

In `Pool` , it will have a field named `assets` ,which contains a list of `assets`

In [2]:
mortgage1 = ["Mortgage"
            ,{"originBalance": 12000.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 40
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 10000.0
            ,"currentRate": 0.075
            ,"remainTerm": 30
            ,"status": "current"}]

mortgage2 = ["Mortgage"
            ,{"originBalance": 800.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 25
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 600.0
            ,"currentRate": 0.075
            ,"remainTerm": 15
            ,"status": "current"}]

myPool = {'assets':[ mortgage1, mortgage2  ],
         'cutoffDate':"2022-03-01"}

In [6]:
r = localAPI.runPool(myPool
                  ,poolAssump=("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                    ,None
                                    ,None)
                  ,read=True)

r['PoolConsol']['flow'].head(5)

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-03-01,9542.79,345.80,61.80,0,7.62,0,7.62,0.075,None,None,345.80,0,0,7.62,0,7.62
2022-04-01,9186.98,347.68,59.58,0,8.13,0,8.13,0.075,None,None,693.48,0,0,15.75,0,15.75
2022-05-01,8829.85,349.56,57.36,0,7.57,0,7.57,0.075,None,None,1043.04,0,0,23.32,0,23.32
2022-06-01,8470.88,351.45,55.13,0,7.52,0,7.52,0.075,None,None,1394.49,0,0,30.84,0,30.84
2022-07-01,8110.54,353.35,52.89,0,6.99,0,6.99,0.075,None,None,1747.84,0,0,37.83,0,37.83


#### Cutoff day

`Cutoff day` means the day after which assets's cashflow belongs to the pool

In [7]:
myPool = {'assets':[ mortgage1, mortgage2  ],
         'cutoffDate':"2023-03-01"}

r = localAPI.runPool(myPool
                  ,poolAssump=("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                    ,None
                                    ,None)
                  ,read=True)

r['PoolConsol']['flow'].head()

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2023-03-01,5168.39,368.94,34.60,0,4.26,0,4.26,0.075,None,None,368.94,0,0,4.26,0,4.26
2023-04-01,4834.54,329.45,32.27,0,4.40,0,4.40,0.075,None,None,698.39,0,0,8.66,0,8.66
2023-05-01,4499.31,331.24,30.19,0,3.99,0,3.99,0.075,None,None,1029.63,0,0,12.65,0,12.65
2023-06-01,4162.45,333.03,28.09,0,3.83,0,3.83,0.075,None,None,1362.66,0,0,16.48,0,16.48
2023-07-01,3824.19,334.83,25.99,0,3.43,0,3.43,0.075,None,None,1697.49,0,0,19.91,0,19.91


#### Mixed Asset Pool

User can build multiple pools with different asset type

In [9]:
receivable1 = ["Invoice"
                  ,{"start":"2024-04-01","originBalance":2000
                    ,"originAdvance":1500,"dueDate":"2024-06-01"}
                  ,{"status":"Current"}]

myMixedPool = {"invoicePool": {'assets':[receivable1],}
              ,"mortgagePool": {'assets':[mortgage1]}
              ,'cutoffDate':"2022-03-01"}

Without any stress assumption, the return result will be a map, which keys are the pool name

In [10]:
r = localAPI.runPool(myMixedPool,read=True)

r['invoicePool']['flow'].head()

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-04-01,2000,0,0,0,0,0,0,0,0,0,0,0,0
2024-06-01,0,0,2000,0,0,0,0,0,0,0,0,0,0


In [11]:
r['mortgagePool']['flow'].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-03-01,9082.00,307.91,58.68,0,0,0,0,0.075,None,None,307.91,0,0,0,0,0
2022-04-01,8772.17,309.83,56.76,0,0,0,0,0.075,None,None,617.74,0,0,0,0,0
2022-05-01,8460.40,311.77,54.82,0,0,0,0,0.075,None,None,929.51,0,0,0,0,0
2022-06-01,8146.68,313.72,52.87,0,0,0,0,0.075,None,None,1243.23,0,0,0,0,0
2022-07-01,7831.00,315.68,50.91,0,0,0,0,0.075,None,None,1558.91,0,0,0,0,0


Set assumptions to the pools
* passing a tuple with key ``ByPoolID`` to pool Assumption
* a map with key equals to the name of pool

In [12]:
mPoolAssump = ("Pool",
               ("Mortgage",{"CDR":0.01},None,None,None)
                ,None
                ,None)

iPoolAssump = ("Pool"
                ,("Receivable", {"CDR":0.01}, None, None)
                ,None
                ,None)


r = localAPI.runPool(myMixedPool
                  ,poolAssump = ("ByPoolId",
                                    {"invoicePool":iPoolAssump
                                    ,"mortgagePool":mPoolAssump}
                                    )
                  ,read=True)

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [16]:
r['invoicePool']['flow'].head(5)

,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-04-01,2000,0,0.00,0,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-06-01,0,0,1996.65,0,3.35,0,3.35,0,0,0,3.35,0,3.35


In [17]:
r['mortgagePool']['flow'].head(5)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-03-01,9059.53,307.14,58.54,0,7.22,0,7.22,0.075,None,None,307.14,0,0,7.22,0,7.22
2022-04-01,8743.01,308.80,56.57,0,7.72,0,7.72,0.075,None,None,615.94,0,0,14.94,0,14.94
2022-05-01,8425.32,310.48,54.59,0,7.21,0,7.21,0.075,None,None,926.42,0,0,22.15,0,22.15
2022-06-01,8105.99,312.15,52.61,0,7.18,0,7.18,0.075,None,None,1238.57,0,0,29.33,0,29.33
2022-07-01,7785.46,313.84,50.62,0,6.69,0,6.69,0.075,None,None,1552.41,0,0,36.02,0,36.02
